## CNN Emotion Detector

##### In this notebook, we will focus on building and training a Convolutional Neural Network (CNN) model to analyze facial expressions in images. 
##### The CNN is a type of deep learning model that is particularly good at processing images. 
##### We will train our CNN to recognize different emotions based on facial expressions.

### Import the necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from mtcnn import MTCNN
from keras.models import load_model
import cv2

### Data Collection
We load the FER2013 dataset and preprocess it. This involves converting the pixel values from strings to integers, reshaping the data into the original image shape, and normalizing the pixel values.

In [ ]:
# Load the data
data = pd.read_csv('fer2013.csv')

# Convert pixels to a numpy array
data['pixels'] = data['pixels'].apply(lambda pixel_sequence: np.fromstring(pixel_sequence, sep=' '))

# Reshape and normalize the data
X = np.vstack(data['pixels'].values)
X = X.reshape(-1, 48, 48, 1)
X = X.astype('float32')
X /= 255.0

# Convert labels to a numpy array
y = data['emotion'].values

### Split the data 
We split the data into a training set and a test set. This allows us to evaluate the performance of our model on unseen data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Define and Train the model 
We define our CNN model for facial expression recognition and train it on our preprocessed data.

In [ ]:
# Initialize the model
model = Sequential()

# Add a convolutional layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))

# Add a pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another convolutional layer
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add another pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the tensor output from the previous layer
model.add(Flatten())

# Add a fully connected layer
model.add(Dense(units=128, activation='relu'))

# Add the output layer
model.add(Dense(units=7, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

### Save the model 
After training, we save our model so that we can use it later without having to retrain it.

In [ ]:
model.save('expression_model.h5')

### Load the model and use it for prediction

In [ ]:
# Load the face detection model
detector = MTCNN()

# Load the facial expression recognition model
expression_model = load_model('expression_model.h5')

# Load the painting
img = cv2.imread('your_painting.jpg')

# Detect faces in the painting
results = detector.detect_faces(img)

for result in results:
    # Get the bounding box coordinates
    x1, y1, width, height = result['box']
    x2, y2 = x1 + width, y1 + height

    # Crop the face from the painting
    face = img[y1:y2, x1:x2]

    # Preprocess the face
    face = cv2.resize(face, (48, 48))
    face = face.astype('float32') / 255
    face = np.expand_dims(face, axis=0)

    # Predict the facial expression
    prediction = expression_model.predict(face)
    emotion = np.argmax(prediction[0])

    # Print the predicted emotion
    print(emotion)